In [8]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [13]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None

In [80]:
def part_text(part,start,end):
    part_list = []
    for i in part[start:end]:
        part_list.append(i.get_text(strip=True).replace('\r', '').replace('\n', '').strip())
        part_text = '\n'.join(part_list)
    return part_text
        

In [97]:
link = 'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-12576-TB-TCHQ-2014-ket-qua-phan-loai-phoi-lieu-de-nau-thuy-tinh-men-xuat-nhap-khau-254591.aspx'
print(link)
response2 = requests.get(link.strip())
soup = BeautifulSoup(response2.text, 'html.parser')
content1 = soup.find('div', class_='content1')
tables = content1.find_all('table')

# lấy số văn bản / ngày
try:
    paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
    for para in paragraphs:
        text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
        if 'Số:' in text:
            số = text.replace("Số:", "").strip()
            # print(số)
            break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
    else:
        print("Không tìm thấy đoạn văn bản chứa 'Số'.")
except AttributeError as e:
        print(f"Lỗi: {e}")
try:
    paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
    for para in paragraphs:
        text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
        if 'ngày' in text or 'Ngày' in text:
            ngày = normalize_date(text)
            # print(ngày)
            break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
    else:
        print("Không tìm thấy đoạn văn bản chứa 'ngày'.")
except AttributeError as e:
    print(f"Lỗi: {e}")
info_list = [link.strip(),số,ngày]

# lấy part 1
part1 = tables[1].find_all('p')       
part1_1_pattern = re.compile(r"1\.\s*[Tt]ên\s*[Hh]àng")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    matches = part1_1_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    if matches:
        part1_1_index = part1.index(p_tag)
part1_2_pattern = re.compile(r"2\.\s*Đơn\s*vị")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    matches = part1_2_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    if matches:
        part1_2_index = part1.index(p_tag)
part1_3_pattern = re.compile(r"3\.\s*Số")
part1_3_pattern_2 = re.compile(r"3\.\s*Tờ\s*khai")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    matches = part1_3_pattern.findall(text) or part1_3_pattern_2.findall(text) # Tìm tất cả các dãy số khớp với pattern trong văn bản
    if matches:
        part1_3_index = part1.index(p_tag)

part1_4_pattern = re.compile(r"4\.\s*Tóm\s*tắt")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    matches = part1_4_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    if matches:
        part1_4_index = part1.index(p_tag)

part1_5_pattern = re.compile(r"5\.\s*Kết\s*quả")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    matches = part1_5_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    if matches:
        part1_5_index = part1.index(p_tag)

part_1_text = part_text(part1,part1_1_index,part1_2_index)
part_2_text = part_text(part1,part1_2_index,part1_3_index)
part_3_text = part_text(part1,part1_3_index,part1_4_index)
part_4_text = part_text(part1,part1_4_index,part1_5_index)

pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
for p_tag in part1[part1_5_index:]:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    if matches:
        part_5_text = matches[-1]

info_list.append(part_1_text)
info_list.append(part_2_text)
info_list.append(part_3_text)
info_list.append(part_4_text)
info_list.append(part_5_text)

print(info_list)


https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-12576-TB-TCHQ-2014-ket-qua-phan-loai-phoi-lieu-de-nau-thuy-tinh-men-xuat-nhap-khau-254591.aspx
['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-12576-TB-TCHQ-2014-ket-qua-phan-loai-phoi-lieu-de-nau-thuy-tinh-men-xuat-nhap-khau-254591.aspx', '12576/TB-TCHQ', '15-10-2014', '1. Tên hàng theo khai báo: Phối liệu để  nấu thủy tinh men Ceramic Frit UF- T39', '2. Đơn vị nhập khẩu:Công ty cổ phần  Gạch men V.T.C; Địa chỉ: Lô 9, Khu công nghiệp Gò Dầu, Huyện Long Thành, Tỉnh  Đồng Nai; Mã số thuế: 3600723398', '3. Số, ngày tờ khai hải quan:  10010414600/A12 ngày 20/8/2014 tại Chi cục Hải quan Long Thành - Cục Hải quan  Tỉnh Đồng Nai', '4, Tóm tắt mô tả và đặc tính hàng hóa:  Men đã được Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO251,2%, CaO 23,4%, TiO217,1%, Al2O33.4 %Na2O  2,4%, K2O 1,2%, ZrO20,1%, MgO 0,8%, Fe2O30,4%', '3207.20.10']


In [98]:
part1

[<p style="margin-bottom:6.0pt"><b>1. Tên hàng theo khai báo</b>: Phối liệu để
   nấu thủy tinh men Ceramic Frit UF- T39</p>,
 <p style="margin-bottom:6.0pt"><b>2. Đơn vị nhập khẩu:</b> Công ty cổ phần
   Gạch men V.T.C; Địa chỉ: Lô 9, Khu công nghiệp Gò Dầu, Huyện Long Thành, Tỉnh
   Đồng Nai; Mã số thuế: 3600723398</p>,
 <p style="margin-bottom:6.0pt"><b>3. Số, ngày tờ khai hải quan</b>:
   10010414600/A12 ngày 20/8/2014 tại Chi cục Hải quan Long Thành - Cục Hải quan
   Tỉnh Đồng Nai</p>,
 <p style="margin-bottom:6.0pt"><b>4, Tóm tắt mô tả và đặc tính hàng hóa</b>:
   Men đã được Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO<sub>2</sub>
   51,2%, CaO 23,4%, TiO<sub>2</sub> 17,1%, Al<sub>2</sub>O<sub>3</sub> 3.4 %Na<sub>2</sub>O
   2,4%, K<sub>2</sub>O 1,2%, ZrO<sub>2</sub> 0,1%, MgO 0,8%, Fe<sub>2</sub>O<sub>3</sub>
   0,4%</p>,
 <p style="margin-bottom:6.0pt"><b><span lang="EN-GB">5. Kết quả phân loại:</span></b></p>,
 <p style="margin-bottom:6.0pt">Tên thương mại: Ceramic Fri

In [93]:
part1_3_pattern = re.compile(r"3\.\s*Số")
part1_3_pattern_2 = re.compile(r"3\.\s*Tờ\s*khai")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    print(text)
    matches = part1_3_pattern.findall(text) or part1_3_pattern_2.findall(text) # Tìm tất cả các dãy số khớp với pattern trong văn bản
    print(matches)
    if matches:
        part1_3_index = part1.index(p_tag)
        print(part1_3_index)

CÔNG CHỨC PTPLNguyễn Ngọc Huân
[]
GIÁM ĐỐC TRUNG TÂM PHÂN TÍCH PHÂN LOẠIHÀNG HOÁ XUẤT KHẨU, NHẬP KHẨU MIỀN BẮCNguyễn Hữu Xuân
[]


In [83]:
part1_4_pattern = re.compile(r"4\.\s*Tóm\s*tắt")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    print(text)
    matches = part1_4_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    print(matches)
    if matches:
        part1_4_index = part1.index(p_tag)
        print(part1_4_index)

1. Tên hàng theo khai báo:Vải thun (theokết quảchứng
  thư giám định tìnhtrạng số 15G10ND00496-301 ngày 18/9/2015) hàng chưa qua sử dụng. (Mục 1 Phụ lục Tờ khai)
[]
2. Đơnvị nhập khẩu:Công ty
  cổ phần Thiết bị và Vật tư Miền Nam - Địa chỉ: 259 Lê Văn Lương, phường Tân
  Quy, Quận 7, TP. Hồ Chí Minh. MST: 0312057516.
[]
3. Tờ khai số:10059048581/A11 ngày 14/10/2015 đăng ký tại Chi cục HQ CK Cảng Cái
  Mép-Cục Hải quan tỉnh BàRịa -
  Vũng Tàu.
[]
4. Tóm tắt mô tả và đặc tính hàng hóa:Vải dệt kim sợi ngang từ bông, đã in các sọc màu, trọng lượng vải =
  204 g/m2.
['4. Tóm tắt']
3
5.Kết
  quảphân loại:
[]
Tên gọi theo cấu tạo, công dụng:
  Vải dệt kim sợi ngang từ bông, đã in các sọc màu, trọng lượng vải = 204 g/m2.
[]
thuộc nhóm60.06“Vải dệt
  kim hoặc móc khác”;phân nhóm“-Từ bông”, mã số6006.24.00“-
  -Đã in”tại Danh mục hàng hóa xuất khẩu, nhập khẩu Việt Nam.
[]


In [74]:
part1_4_index

5

In [75]:
part1_5_pattern = re.compile(r"5\.\s*Kết\s*quả")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    print(text)
    matches = part1_5_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    print(matches)
    if matches:
        part1_5_index = part1.index(p_tag)
        print(part1_5_index)

1. Tên hàng theo khai báo:Nhựa coumarone-indene, dạng nguyên sinh (EScron G-90).
[]
2. Đơn vị nhập khẩu:Công
  ty TNHH Việt Nam NOK.
[]
Đ/c: Lô 208, KCN Amata, huyện
  Long Bình, tỉnh Đồng Nai.
[]
MST: 3600689323.
[]
3. Số, ngày tờ khai hải quan:12554/NKD05 ngày 01/11/2013 tại Chi cục Hải quan Biên Hòa.
[]
4. Tóm tắt mô tả và đặc tính
  hàng hóa:Nhựa coumarone-indene, dạng nguyên sinh (EScron G-90).
[]
5. Kết quả phân loại:
['5. Kết quả']
6
Tên thương mại: ESCRON G-90.
[]
Tên gọi theo cấu tạo, bản chất,
  công dụng: Nhựa coumarone-indene, dạng hạt.
[]
Ký, mã hiệu, chủng loại:
  RM00-124002.
[]
Nhà sản xuất: Ouchi Shinko
  Chemical Industrial Co., LTD
[]
thuộc Chương 39 "Plastic và
  các sản phẩm bằng plastic", nhóm 39.11 "Nhựa từ dầu mỏ, nhựa
  Coumarone-indene, polyterpen, polysulphua, polysunphon và các sản phẩm khác
  đã nêu trong chú giải 3 của Chương này, chưa được chi tiết hoặc ghi ở nơi
  khác, dạng nguyên sinh", mã số 39.11.10.00 "- Nhựa từ dầu mỏ, nhựa
  cumaron, nhựa inden ho

In [76]:
part1_5_index

6

In [64]:
pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
for p_tag in part1[part1_5_index:]:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    if matches:
        part_5_text = matches[-1]

In [65]:
part_5_text

'3823.19.90'